In [1]:
pip install google-play-scraper

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
from google_play_scraper import Sort, reviews_all
import pandas as pd

# input the app package name directly
app_package = 'org.thoughtcrime.securesms'

# scrape the app reviews
reviews = reviews_all(
    app_package,
    lang='en',
    country='ca',
    sort=Sort.NEWEST,
    count=1000
)

# create a dataframe from the reviews
df = pd.DataFrame(reviews)
df = df.assign(Package_Name=app_package)
df = df[:1000]
df = df.loc[:, ['Package_Name', 'userName', 'content', 'score']]
# print the dataframe
print(df.head())
df.to_csv('Signal reviews.csv', index=False)

                 Package_Name        userName  \
0  org.thoughtcrime.securesms  Jonathan Moore   
1  org.thoughtcrime.securesms      Amanda Lee   
2  org.thoughtcrime.securesms       deb casey   
3  org.thoughtcrime.securesms         D. Casu   
4  org.thoughtcrime.securesms   William Sloma   

                                             content  score  
0  Jolly good. Does what you want and nothing you...      5  
1                                           Nice app      4  
2                                Works really well 👌      5  
3              Lost 1 star for removing SMS support.      4  
4  You stop allowing SMS, you serve no purpose an...      1  


Reference: https://www.linkedin.com/pulse/how-scrape-google-play-reviews-4-simple-steps-using-python-kundi/ 
All other apps were scraped in a similar manner.

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install inflect

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install vader

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sejal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sejal\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import inflect

In [9]:
# Remove punctuations from the reviewcolumn
def punctuations(reviewcolumn):
    return reviewcolumn.str.replace(f"[{string.punctuation}]", '', regex = True)

In [10]:
# Removes special characters and emojies from the reviewcolumn
def characterandemoji(reviewcolumn):
    reviewcolumn = reviewcolumn.str.replace(r'[^\w\s]|_', '',  regex = True)  # Remove special characters
    reviewcolumn = reviewcolumn.str.encode('ascii', 'ignore').str.decode('ascii')  # Remove emojies
    return reviewcolumn

In [11]:
# Convert numbers to words
# I have used the inflect library to convert the numbers to words
def numbertowords(reviewcolumn):
    for i in range(len(reviewcolumn)):
        words = []
        review = str(reviewcolumn[i])
        for match in re.finditer(r'\d+|\D+', review):
            word = match.group(0)
            if word.isdigit():
                words.append(inflect.engine().number_to_words(word))
            elif any(char.isdigit() for char in word):
                parts = re.findall(r'\d+|\D+', word)
                modified = []
                for part in parts:
                    if part.isdigit():
                        modified.append(inflect.engine().number_to_words(part))
                    else:
                        modified.append(part)
                    words.append(''.join(modified))
            else:
                words.append(word)
        reviewcolumn[i] = ''.join(words)
    return reviewcolumn

In [12]:
# Removes extra white spaces
def extraspace(reviewcolumn):
    reviewcolumn = reviewcolumn.str.replace(r'\s+', ' ', regex = True)
    return reviewcolumn

In [13]:
# Changes the reviews in reviewcolumn to lowercase
def lowercase(reviewcolumn):
    reviewcolumn = reviewcolumn.str.lower()
    return reviewcolumn

In [14]:
# Remove stopwords
def removestop(reviewcolumn):
    stopwordslist = stopwords.words('english')
    filteredreviews = []
    for i in range(len(reviewcolumn)):
        words = []
        for word in reviewcolumn[i].split():
            if word not in stopwordslist:
                words.append(word)
        filteredreviews.append(' '.join(words))
    return pd.DataFrame(filteredreviews, columns=['Review'])

In [15]:
# Lemmatizes the words in the reviews of reviewcolumn
def lemmatize(filteredreviews):
    lemmatizer = WordNetLemmatizer()
    lemmatizedreviews = []
    for review in filteredreviews['Review']:
        words = review.split()
        lemmatizedwords = []
        for word in words:
            lemmatizedword = lemmatizer.lemmatize(word)
            lemmatizedwords.append(lemmatizedword)
        lemmatizedreview = ' '.join(lemmatizedwords)
        lemmatizedreviews.append(lemmatizedreview)
    filteredreviews['Review'] = lemmatizedreviews
    return filteredreviews

In [16]:
# Calculates the polarity of each review from Textblob library
def textblobpolarity(reviewcolumn):
    polarities = []
    for review in reviewcolumn['Review']:
        blob = TextBlob(review)
        polarity = blob.sentiment.polarity
        polarities.append(polarity)
    result = pd.DataFrame({'App\'s package name': reviews['Package Name'],'Review': reviewcolumn['Review'],'Polarity': polarities})
    return result

In [17]:
# Calculate the polarity of each review from Vader library
def vaderpolarity(reviewcolumn):
    polarities = []
    for review in reviewcolumn['Review']:
        scores = SentimentIntensityAnalyzer().polarity_scores(review)
        polarities.append(scores['compound'])
    result = pd.DataFrame({'App\'s package name': reviews['Package Name'],'Review': reviewcolumn['Review'],'Polarity': polarities})
    return result

SUD: QKSMS

In [18]:
reviews = pd.read_csv('com.moez.QKSMS.csv')
reviewcolumn = reviews['Review']
# Print first 15 rows of review column before pre-processing
print("Before pre-processing: ")
print(reviewcolumn.head(15))

Before pre-processing: 
0     Crashing on me on p, Not sure why it is doing ...
1                                           Easy to use
2     I think Indians can't make trully free and ope...
3     Why does this app require camer accesess? The ...
4                                           I love y ou
5     Thought I would give this a try. Nice UI and c...
6     So far so good... Easier to use and configure ...
7     Awesome alternative to onboard text messaging ...
8                                  material you? plans?
9     Worst app ever. Won't let you add sender to co...
10    I love the fact it blocks scammers with should...
11    Sometimes I want to delete all the posts in a ...
12                                Please update the app
13    Installed because Signal will no longer suppor...
14    I am having an issue with this texting app. It...
Name: Review, dtype: object


TASK 2: PRE-PROCESS YOUR TEXT

In [19]:
print("After removing punctuations:")
reviewcolumn = punctuations(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing special characters and emojies:")
reviewcolumn = characterandemoji(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting number to text:")
reviewcolumn = numbertowords(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing extra white spaces:")
reviewcolumn = extraspace(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting to lowercase:")
reviewcolumn = lowercase(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing stopwords:")
reviewcolumn = removestop(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After Lemmatizing:")
lemmatizedreviews = lemmatize(reviewcolumn)
reviewcolumn['Review'] = lemmatizedreviews
print(reviewcolumn.head(15))

After removing punctuations:
0     Crashing on me on p Not sure why it is doing t...
1                                           Easy to use
2     I think Indians cant make trully free and open...
3     Why does this app require camer accesess The t...
4                                           I love y ou
5     Thought I would give this a try Nice UI and cu...
6     So far so good Easier to use and configure tha...
7     Awesome alternative to onboard text messaging ...
8                                    material you plans
9     Worst app ever Wont let you add sender to cont...
10    I love the fact it blocks scammers with should...
11    Sometimes I want to delete all the posts in a ...
12                                Please update the app
13    Installed because Signal will no longer suppor...
14    I am having an issue with this texting app Its...
Name: Review, dtype: object

After removing special characters and emojies:
0     Crashing on me on p Not sure why it is doing t...

In [20]:
print("After Pre-processing:")
print(reviewcolumn.head(15))

After Pre-processing:
                                               Review
0            crashing p sure sudden didnt make change
1                                            easy use
2   think indian cant make trully free opensource ...
3   app require camer accesess type forcfuly avoid...
4                                             love ou
5   thought would give try nice ui customization n...
6     far good easier use configure native app mobile
7     awesome alternative onboard text messaging apps
8                                       material plan
9   worst app ever wont let add sender contact bas...
10  love fact block scammer answer pairing one sta...
11  sometimes want delete post long conversation e...
12                                  please update app
13  installed signal longer support sm qksms allow...
14  issue texting app delivering text message big ...


TASK 3: SENTIMENT ANALYSIS

In [21]:
# Polarity tables for QKSMS
print("The Polarity table from Textblob:")
print(textblobpolarity(reviewcolumn))
print("")
print("The Polarity table from Vader:")
print(vaderpolarity(reviewcolumn))

The Polarity table from Textblob:
    App's package name                                             Review  \
0       com.moez.QKSMS           crashing p sure sudden didnt make change   
1       com.moez.QKSMS                                           easy use   
2       com.moez.QKSMS  think indian cant make trully free opensource ...   
3       com.moez.QKSMS  app require camer accesess type forcfuly avoid...   
4       com.moez.QKSMS                                            love ou   
..                 ...                                                ...   
995     com.moez.QKSMS  texting app use one free version good enough l...   
996     com.moez.QKSMS                                  unable send video   
997     com.moez.QKSMS                                nice excellent apps   
998     com.moez.QKSMS  getting message contact luckily run messaging ...   
999     com.moez.QKSMS  alright dont like used enable call control app...   

     Polarity  
0    0.250000  
1    0.43

V. Comparison between the polarity tables
Analyzing the polarity tables from Textblob and Vader, it can be seen that the scores calculated by both the libraries are mostly similar. 
The similarities and differences can be observed from the differences between the numbers of the corresponding reviews. 
Similarity:
For example: review 1 has polarity 0.4333 by Textblob and 0.4404 by Vader which is almost similar, review 4 has polarity 0.5000 by Textblob and 0.6369 by Vader which is almost similar. 
Difference:
There are slight variations which can be observed, as in the case of review 999 where Textblob has calcuated a polarity of 0, that is neutral while Vader calculated it as -0.3421. The differences are due to the way these 2 libraries calculate the sentiment of the reviews. 
Therefore, the overall comparison between the 2 libraries depends on the review being analyzed. Vader handles individual words and expressions and also takes into consideration emojies, punctuations and capitalization but as all of these have already been removed during the preprocessing, so, it is not really beneficial in our context. In our case, Textblob might be a better option because it calculates the overall sentiment of the sentence. 

Similar app 1: Textra SMS

In [22]:
reviews = pd.read_csv('com.textra.csv')
reviewcolumn = reviews['Review']
# Print first 15 rows of review column before pre-processing
print("Before pre-processing: ")
print(reviewcolumn.head(15))

Before pre-processing: 
0     Forcing you to use the app as your default lit...
1     I would love the ability to restrict searches ...
2     I wanted to send group texts and have the repl...
3     I've been using this app for as long as I've h...
4     So far, so good! I had been using the pre-inst...
5                                           Works Great
6     This is easy to use, clear and well-engineered...
7     I love this app but recently upgraded my phone...
8     Great features but the one I want doesn't work...
9     I've used Textra for a long time. Let me tell ...
10    It has a fundamental flaw which the developers...
11    So far I like it but the whole reason I got th...
12    I had heard good things about Textra, but the ...
13    1 tiny ad a few times a week at the top. Don't...
14                               Very useful and stable
Name: Review, dtype: object


In [23]:
print("After removing punctuations:")
reviewcolumn = punctuations(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing special characters and emojies:")
reviewcolumn = characterandemoji(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting number to text:")
reviewcolumn = numbertowords(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing extra white spaces:")
reviewcolumn = extraspace(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting to lowercase:")
reviewcolumn = lowercase(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing stopwords:")
reviewcolumn = removestop(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After Lemmatizing:")
lemmatizedreviews = lemmatize(reviewcolumn)
reviewcolumn['Review'] = lemmatizedreviews
print(reviewcolumn.head(15))

After removing punctuations:
0     Forcing you to use the app as your default lit...
1     I would love the ability to restrict searches ...
2     I wanted to send group texts and have the repl...
3     Ive been using this app for as long as Ive had...
4     So far so good I had been using the preinstall...
5                                           Works Great
6     This is easy to use clear and wellengineered J...
7     I love this app but recently upgraded my phone...
8     Great features but the one I want doesnt work ...
9     Ive used Textra for a long time Let me tell yo...
10    It has a fundamental flaw which the developers...
11    So far I like it but the whole reason I got th...
12    I had heard good things about Textra but the a...
13    1 tiny ad a few times a week at the top Dont e...
14                               Very useful and stable
Name: Review, dtype: object

After removing special characters and emojies:
0     Forcing you to use the app as your default lit...

In [24]:
print("After Pre-processing:")
print(reviewcolumn.head(15))

After Pre-processing:
                                               Review
0   forcing use app default literally defeat purpo...
1   would love ability restrict search thread addr...
2   wanted send group text reply come app able col...
3   ive using app long ive dmart phone way make me...
4   far good using preinstalled google messenger y...
5                                          work great
6   easy use clear wellengineered simple efficient...
7   love app recently upgraded phone seems acting ...
8   great feature one want doesnt work regularly b...
9   ive used textra long time let tell there nothi...
10  fundamental flaw developer seem bothered fixin...
11  far like whole reason got app bc seen video sa...
12  heard good thing textra amount text never rece...
13  one tiny ad time week top dont even notice twe...
14                                      useful stable


In [25]:
# Polarity tables for Textra
print("The Polarity table from Textblob:")
print(textblobpolarity(reviewcolumn))
print("")
print("The Polarity table from Vader:")
print(vaderpolarity(reviewcolumn))

The Polarity table from Textblob:
    App's package name                                             Review  \
0           com.textra  forcing use app default literally defeat purpo...   
1           com.textra  would love ability restrict search thread addr...   
2           com.textra  wanted send group text reply come app able col...   
3           com.textra  ive using app long ive dmart phone way make me...   
4           com.textra  far good using preinstalled google messenger y...   
..                 ...                                                ...   
995         com.textra  always come back app great able react message ...   
996         com.textra  textra simple use easier delete unwanted text ...   
997         com.textra                            preferred messaging app   
998         com.textra  great app better stock sm one issue run cant s...   
999         com.textra  app good wouldnt give five star doesnt receive...   

     Polarity  
0   -0.166667  
1    0.50

V. Comparison between the polarity tables 
Analyzing the polarity tables from Textblob and Vader, it can be seen that the scores calculated by both the libraries are mostly similar. 
The similarities and differences can be observed from the differences between the numbers of the corresponding reviews. Similarity: 
For example: review 1 has polarity 0.5 by Textblob and 0.4939 by Vader which is almost similar and shows that both the reviews are positive. Review 998 has polarity 0.65 by Textblob and 0.7279 by Vader which is again similar. 
Difference: 
For example: review 0 has polarity -0.166667 by Textblob and -0.6369 by Vader which is significantly different. It can be observed that Vader is more negative than Textblob. The differences are due to the way these 2 libraries calculate the sentiment of the reviews. 
Therefore, the overall comparison between the 2 libraries depends on the review being analyzed. Vader handles individual words and expressions and also takes into consideration emojies, punctuations and capitalization but as all of these have already been removed during the preprocessing, it is not really beneficial in our context. In our case, Textblob might be a better option because it calculates the overall sentiment of the sentence.

Similar app 2: Pulse SMS

In [26]:
reviews = pd.read_csv('xyz.klinker.messenger.csv')
reviewcolumn = reviews['Review']
# Print first 15 rows of review column before pre-processing
print("Before pre-processing: ")
print(reviewcolumn.head(15))

Before pre-processing: 
0                                          It is better
1     Since new update, app doesn't show new message...
2     Everything was great for the past 3 years with...
3     I have been using this app for a few years and...
4     Used to be great, but has gone downhill recent...
5     love the pulse music app! very customizable. l...
6     Been using it for awhile with no issues, excep...
7     I tried this app out and canceled the same day...
8                                            Mulchandsk
9     I have had for years, and used to work fine un...
10    Buggy after the last update. New texts are not...
11    No app for wear os so can't get messages on ga...
12    This used to be awesome, but then they put the...
13                         Can't sync phone and tablet,
14                                                 nice
Name: Review, dtype: object


In [27]:
print("After removing punctuations:")
reviewcolumn = punctuations(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing special characters and emojies:")
reviewcolumn = characterandemoji(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting number to text:")
reviewcolumn = numbertowords(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing extra white spaces:")
reviewcolumn = extraspace(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting to lowercase:")
reviewcolumn = lowercase(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing stopwords:")
reviewcolumn = removestop(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After Lemmatizing:")
lemmatizedreviews = lemmatize(reviewcolumn)
reviewcolumn['Review'] = lemmatizedreviews
print(reviewcolumn.head(15))

After removing punctuations:
0                                          It is better
1     Since new update app doesnt show new messages ...
2     Everything was great for the past 3 years with...
3     I have been using this app for a few years and...
4     Used to be great but has gone downhill recentl...
5     love the pulse music app very customizable lov...
6     Been using it for awhile with no issues except...
7     I tried this app out and canceled the same day...
8                                            Mulchandsk
9     I have had for years and used to work fine unt...
10    Buggy after the last update New texts are not ...
11    No app for wear os so cant get messages on gal...
12    This used to be awesome but then they put the ...
13                           Cant sync phone and tablet
14                                                 nice
Name: Review, dtype: object

After removing special characters and emojies:
0                                          It is better

In [28]:
print("After Pre-processing:")
print(reviewcolumn.head(15))

After Pre-processing:
                                               Review
0                                              better
1   since new update app doesnt show new message o...
2   everything great past three year pulse current...
3   using app year last day message longer properl...
4   used great gone downhill recently recent updat...
5   love pulse music app customizable love shuffle...
6   using awhile issue except seems like new messa...
7   tried app canceled day said would charge charg...
8                                          mulchandsk
9   year used work fine recent update sending new ...
10  buggy last update new text highlighted brought...
11           app wear o cant get message galaxy watch
12  used awesome put scheduled message behind pay ...
13                             cant sync phone tablet
14                                               nice


In [29]:
# Polarity tables for Pulse SMS
print("The Polarity table from Textblob:")
print(textblobpolarity(reviewcolumn))
print("")
print("The Polarity table from Vader:")
print(vaderpolarity(reviewcolumn))

The Polarity table from Textblob:
        App's package name                                             Review  \
0    xyz.klinker.messenger                                             better   
1    xyz.klinker.messenger  since new update app doesnt show new message o...   
2    xyz.klinker.messenger  everything great past three year pulse current...   
3    xyz.klinker.messenger  using app year last day message longer properl...   
4    xyz.klinker.messenger  used great gone downhill recently recent updat...   
..                     ...                                                ...   
995  xyz.klinker.messenger                                               nice   
996  xyz.klinker.messenger  somewhat convoluted trying install another and...   
997  xyz.klinker.messenger         great far give five star maybe used option   
998  xyz.klinker.messenger  unistalling app continues make life difficult ...   
999  xyz.klinker.messenger  move new phone get anyone help guess add paid .

V. Comparison between the polarity tables 
Analyzing the polarity tables from Textblob and Vader, it can be seen that the scores calculated by both the libraries are mostly similar. 
The similarities and differences can be observed from the differences between the numbers of the corresponding reviews. Similarity: 
For example: review 0 has polarity 0.5 by Textblob and 0.4404 by Vader which is almost similar and shows that both the reviews are positive. Review 1 has polarity 0.174242 by Textblob and 0.2023 by Vader which is again similar. 
Difference: 
For example: review 998 has polarity -0.166667 by Textblob and 0.0258 by Vader which is significantly different. Textblob has given a negative polarity which indicates a negative review while Vader has given a positive review which shows a positive review about the app. The differences are due to the way these 2 libraries calculate the sentiment of the reviews. 
Therefore, the overall comparison between the 2 libraries depends on the review being analyzed. Vader handles individual words and expressions and also takes into consideration emojies, punctuations and capitalization but as all of these have already been removed during the preprocessing, so, it is not really beneficial in our case. In our case, Textblob might be a better option because it calculates the overall sentiment of the sentence.

Competitor app 1: Signal Private Messenger

In [30]:
reviews = pd.read_csv('org.thoughtcrime.securesms.csv')
reviewcolumn = reviews['Review']
# Print first 15 rows of review column before pre-processing
print("Before pre-processing: ")
print(reviewcolumn.head(15))

Before pre-processing: 
0     Jolly good. Does what you want and nothing you...
1                                              Nice app
2                                Works really well ðŸ‘Œ
3                 Lost 1 star for removing SMS support.
4     You stop allowing SMS, you serve no purpose an...
5     My son is active duty Navy, 3rd generation mil...
6     I utilized this application for a little while...
7     Very useful. Why do I need a phone number. See...
8                                 Keep up the good work
9     Every phone supports sms, I wouldn't care if i...
10                                      Why remove SMS?
11                                    Good happy g chat
12                                              Perfect
13    With the fact that you're ditching SMS text me...
14                                         Ok very good
Name: Review, dtype: object


In [31]:
print("After removing punctuations:")
reviewcolumn = punctuations(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing special characters and emojies:")
reviewcolumn = characterandemoji(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting number to text:")
reviewcolumn = numbertowords(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing extra white spaces:")
reviewcolumn = extraspace(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting to lowercase:")
reviewcolumn = lowercase(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing stopwords:")
reviewcolumn = removestop(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After Lemmatizing:")
lemmatizedreviews = lemmatize(reviewcolumn)
reviewcolumn['Review'] = lemmatizedreviews
print(reviewcolumn.head(15))

After removing punctuations:
0     Jolly good Does what you want and nothing you ...
1                                              Nice app
2                                Works really well ðŸ‘Œ
3                  Lost 1 star for removing SMS support
4     You stop allowing SMS you serve no purpose any...
5     My son is active duty Navy 3rd generation mili...
6     I utilized this application for a little while...
7     Very useful Why do I need a phone number Seems...
8                                 Keep up the good work
9     Every phone supports sms I wouldnt care if it ...
10                                       Why remove SMS
11                                    Good happy g chat
12                                              Perfect
13    With the fact that youre ditching SMS text mes...
14                                         Ok very good
Name: Review, dtype: object

After removing special characters and emojies:
0     Jolly good Does what you want and nothing you ...

In [32]:
print("After Pre-processing:")
print(reviewcolumn.head(15))

After Pre-processing:
                                               Review
0                   jolly good want nothing dont want
1                                            nice app
2                                    work really well
3                   lost one star removing sm support
4   stop allowing sm serve purpose anymore uninsta...
5   son active duty navy threerd generation milita...
6   utilized application little functioned fine no...
7   useful need phone number seems unnecessary dep...
8                                      keep good work
9   every phone support sm wouldnt care paid featu...
10                                          remove sm
11                                  good happy g chat
12                                            perfect
13  fact youre ditching sm text messaging make lon...
14                                            ok good


In [33]:
# Polarity tables for Signal private messenger
print("The Polarity table from Textblob:")
print(textblobpolarity(reviewcolumn))
print("")
print("The Polarity table from Vader:")
print(vaderpolarity(reviewcolumn))

The Polarity table from Textblob:
             App's package name  \
0    org.thoughtcrime.securesms   
1    org.thoughtcrime.securesms   
2    org.thoughtcrime.securesms   
3    org.thoughtcrime.securesms   
4    org.thoughtcrime.securesms   
..                          ...   
995  org.thoughtcrime.securesms   
996  org.thoughtcrime.securesms   
997  org.thoughtcrime.securesms   
998  org.thoughtcrime.securesms   
999  org.thoughtcrime.securesms   

                                                Review  Polarity  
0                    jolly good want nothing dont want      0.70  
1                                             nice app      0.60  
2                                     work really well      0.20  
3                    lost one star removing sm support      0.00  
4    stop allowing sm serve purpose anymore uninsta...      0.00  
..                                                 ...       ...  
995  privacy dont ever fooled using app private ins...      0.00  
996      

Competitor app 2: Google messages

In [34]:
reviews = pd.read_csv('com.google.android.apps.messaging.csv')
reviewcolumn = reviews['Review']
# Print first 15 rows of review column before pre-processing
print("Before pre-processing: ")
print(reviewcolumn.head(15))

Before pre-processing: 
0                                              All good
1                             Google is the greatest!!!
2           It's a good, functional app. Does it's job.
3                    I love the convient options given.
4     Great app that allows me to edit, send, and sa...
5                                              Good app
6                                                  Good
7                                    Timely application
8                                           Amazing App
9     It's a text message app....what praise do you ...
10    It's quick & easy to set up & use also it does...
11    Messages don't send or deliver. Creating commu...
12                               Excellent service good
13    I am not hearing any notifications from this a...
14                                         Slow loading
Name: Review, dtype: object


In [35]:
print("After removing punctuations:")
reviewcolumn = punctuations(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing special characters and emojies:")
reviewcolumn = characterandemoji(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting number to text:")
reviewcolumn = numbertowords(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing extra white spaces:")
reviewcolumn = extraspace(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After converting to lowercase:")
reviewcolumn = lowercase(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After removing stopwords:")
reviewcolumn = removestop(reviewcolumn)
print(reviewcolumn.head(15))
print("")
print("After Lemmatizing:")
lemmatizedreviews = lemmatize(reviewcolumn)
reviewcolumn['Review'] = lemmatizedreviews
print(reviewcolumn.head(15))

After removing punctuations:
0                                              All good
1                                Google is the greatest
2                Its a good functional app Does its job
3                     I love the convient options given
4     Great app that allows me to edit send and save...
5                                              Good app
6                                                  Good
7                                    Timely application
8                                           Amazing App
9         Its a text message appwhat praise do you want
10    Its quick  easy to set up  use also it doesnt ...
11    Messages dont send or deliver Creating communi...
12                               Excellent service good
13    I am not hearing any notifications from this a...
14                                         Slow loading
Name: Review, dtype: object

After removing special characters and emojies:
0                                              All good

In [36]:
print("After Pre-processing:")
print(reviewcolumn.head(15))

After Pre-processing:
                                               Review
0                                                good
1                                     google greatest
2                             good functional app job
3                          love convient option given
4   great app allows edit send save message truly ...
5                                            good app
6                                                good
7                                  timely application
8                                         amazing app
9                    text message appwhat praise want
10                quick easy set use also doesnt buff
11  message dont send deliver creating communicati...
12                             excellent service good
13  hearing notification app im getting message ev...
14                                       slow loading


In [37]:
# Polarity tables for Google messages
print("The Polarity table from Textblob:")
print(textblobpolarity(reviewcolumn))
print("")
print("The Polarity table from Vader:")
print(vaderpolarity(reviewcolumn))

The Polarity table from Textblob:
                    App's package name  \
0    com.google.android.apps.messaging   
1    com.google.android.apps.messaging   
2    com.google.android.apps.messaging   
3    com.google.android.apps.messaging   
4    com.google.android.apps.messaging   
..                                 ...   
995  com.google.android.apps.messaging   
996  com.google.android.apps.messaging   
997  com.google.android.apps.messaging   
998  com.google.android.apps.messaging   
999  com.google.android.apps.messaging   

                                                Review  Polarity  
0                                                 good     0.700  
1                                      google greatest     1.000  
2                              good functional app job     0.700  
3                           love convient option given     0.500  
4    great app allows edit send save message truly ...     0.775  
..                                                 ...     

TASK 4: TOPIC MODELING Using LDA

In [38]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [39]:
import gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

In [40]:
# create a dictionary of the reviews
dictionary = Dictionary([review.split() for review in reviewcolumn['Review']])

# creates a corpus for the dictionary
corpus = [dictionary.doc2bow(review.split()) for review in reviewcolumn['Review']]

# Used the LDA Model according to Assignment instructions
ldamodel = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=10)

for i, topic in ldamodel.show_topics(num_topics=10, num_words=7, formatted=False):
    print(f"Topic {i+1}: {[word[0] for word in topic]}")

Topic 1: ['app', 'great', 'awesome', 'chat', 'change', 'update', 'message']
Topic 2: ['message', 'cant', 'reply', 'code', 'short', 'update', 'bug']
Topic 3: ['nice', 'app', 'excellent', 'use', 'phone', 'easy', 'apps']
Topic 4: ['message', 'get', 'app', 'update', 'dont', 'send', 'annoying']
Topic 5: ['ok', 'bad', 'like', 'update', 'msg', 'app', 'phone']
Topic 6: ['good', 'app', 'best', 'blah', 'work', 'helpful', 'experience']
Topic 7: ['app', 'use', 'easy', 'text', 'super', 'time', 'useful']
Topic 8: ['app', 'love', 'message', 'send', 'notification', 'error', 'video']
Topic 9: ['message', 'app', 'update', 'issue', 'phone', 'open', 'send']
Topic 10: ['message', 'app', 'text', 'time', 'sometimes', 'love', 'receive']


I. All my apps have 1000+ reviews but I chose Google messages for Topic modelling as it has highest number of reviews available on Google play store.

II. The topics have been extracted above.

III. The software feature clusters 1, 2, 5, 7 obtained in Assignment 1 and 2 for the SUD are irrelevant to the topics obtained above for the Competitor app Google messages.
Relevance: 
Topic 8 is relevant to cluster 4. Cluster 4 is about the requirement of notifications for users. Similarly, Topic 8 has description about notifications, which makes Google messages a competitor of QKSMS. As per the reviews obtained from Google messages and QKSMS, I manually went through the reviews searching for the reviews about notifications and it can be observed that the users of both the apps want the ability of custom notifications from the app which makes them a competitor. 
Topic 9,10 is relevant to cluster 6 which is about QKSMS being able to send, receive and process SMS messages. From topic 9 and 10, it can be concluded that the topic explains that Google messages is also a messaging app and its basic functionality is being able to send, read and process messages. 
Topic 1 is relevant to cluster 3. Cluster 3 is about custom theme options like automatic night mode. Similarly, topic 1 is also about users wanting more theme options which can be seen from the words "change" and "update". I went through the reviews of Google messages and found that its users want to change themes of the app which is already available in QKSMS making it a competitor app.
On the basis of these relevance between the software feature clusters of QKSMS and the topics obtained for Google messages, it can be concluded that Google messages is a competitor of QKSMS. 
As Google messages is a messaging app, we can see that the word "message" is appearing multiple times and can be removed if we add it to the list of stop words to refine our topics.

IV. As Google apps had a lot of reviews, it was really beneficial to do the data analysis as we had a lot of information. But we were supposed to filter only 1000 reviews for topic modelling. So, I chose the 1000 most recent reviews as with the passage of time, the reviews become irrelevant. So, the size of the data was moderate for initial analysis and we can change it further if needed. Choosing 10 topics out of the 1000 reviews have resulted in a broad variety of topics. If I need to improve the results, I will need to increase the number of reviews to analyse or the number of topics to extract as a starting point for further analysis. Moreover, if we increase the number of reviews for performing the topic modelling, it might lead to capturing fake and spam reviews which will add noise to the topics obtained and might lead to inaccurate results. So, it is a matter of tuning the parameters to get the best possible results. Therefore, the accuracy of the model is dependent on various parameters. 

V. The core functionalities obtained from assignment 1 that are relevant to the topics obtained above are cluster 3 and 6. They are the core functionalities for both the SUD and the competitor app, Google messages. In assignment 2, UR10: "Users require an intuitive design that allow them to focus on what matters." is also added to the core functionalities of QKSMS, which is also relevant to Google messages as Google messages is basically better in terms of providing intuitive design as it has less customisation options and focuses on just sending and receiving SMS/MMS.

TASK 5: RECOMMENDATION

I. Enhancement 1: As per https://github.com/moezbhatti/qksms/issues/1699, users are facing an issue when changing the theme color of the app. Therefore, the transition should be smooth and seamless. There should be no jankiness. I also went through the issues reported on Github and this issue has not been resolved yet. This can be improved by changing the animation code or maybe using different animation library or other solutions. It is in the list of backlog, therefore, it is yet to be completed.
Enhancement 2: As per https://github.com/moezbhatti/qksms/issues/1693, MIUI 11 has been facing issues with the notification sounds. So, the app should be updated to be fully compatible with the notification system in MIUI 11 which is different than other android systems. The issues should be fixed by fixing the errors in the code making it compatible with MIUI 11. I went through the issues reported on Github and this issue has not been resolved yet. Moreover, this bug report has been added to the list of backlog issues, which shows that this enhancement has not been applied yet.

II. Similarity: The persona of Aislynn wants custom ringtones and notifications from QKSMS, which is similar to the users of MIUI 11 who are having issues with the notification system of MIUI 11. 
The persona of Timothy wants customisable theme options for the app and they should be working smoothly according to Enhancement 1 as the users are facing jankiness when changing the theme options of the app.
Difference: There are no such differences between both the enhancements and the personas, they are just a subset of the needs of the personas.